In [1]:
import numpy as np
import pandas as pd

In [2]:
data = [
    ("A", "I", 1),
    ("A", "II", 2),
    ("A", "III", 3),
    ("A", "IV", 4),
    ("B", "I", 3),
    ("B", "II", 2),
    ("B", "III", 1),
    ("B", "IV", 4),
    ("C", "I", 1),
    ("C", "II", 2),
    ("C", "III", 4),
    ("C", "IV", 4)
]

data4 = [
    ("A", "I", 1),
    ("A", "II", 2),
    ("A", "III", 3),
    ("B", "I", 3),
    ("B", "II", 2),
    ("B", "III", 1),
    ("B", "IV", 4),
    ("C", "I", 1),
    ("C", "II", 2),
    ("C", "III", 4),
    ("C", "IV", 4)
]

# Voters	Ranking
data2 = [
    (30, ["A", "B", "C"]),
    ( 1, ["A", "C", "B"]),
    (29, ["B", "A", "C"]),
    (10, ["B", "C", "A"]),
    (10, ["C", "A", "B"]),
    ( 1, ["C", "B", "A"])
]

data3 = [
    (7, ["A", "B", "C", "D"]),
    (5, ["B", "C", "D", "A"]),
    (4, ["D", "B", "C", "A"]),
    (3, ["C", "D", "A", "B"])
]

data5 = [
    (3, ["A", "B", "C"]),
    (2, ["B", "A", "C"]),
    (5, ["C", "A", "B"])
]

In [3]:
df = pd.DataFrame(data, columns=["voter", "candidate", "rank"])
df4 = pd.DataFrame(data4, columns=["voter", "candidate", "rank"])
df2 = pd.DataFrame(data2, columns=["voters", "rank"])
df3 = pd.DataFrame(data3, columns=["voters", "rank"])
df5 = pd.DataFrame(data5, columns=["voters", "rank"])

In [ ]:
asdf = transform(df2, unique_id=True)

In [ ]:
borda(df2)

In [ ]:
copeland(df2)

In [ ]:
dd = load_synthetic_pairwise(ties=True)
print(dd.groupby("selected").count().loc[0])
print(dd.shape[0])

In [ ]:
dd = load_synthetic_election(n_candidates = 7, n_voters = 5000)
dd

In [ ]:
m = Voting(dd).copeland_matrix(n_votes = True)

In [ ]:
Voting(dd).borda()

In [ ]:
from itertools import permutations

def kemeny_young(self, score_matrix = False):
    m = self.copeland_matrix(n_votes = True)
    candidate = self.candidate
    rank = self.rank

    output = []
    for permutation in permutations(list(m)):
        score = 0
        for items in combinations(permutation, 2):
            i_winner, i_loser = items
            value = m.loc[i_winner, i_loser]
            score += value
        output.append([list(permutation), score])

    tmp = pd.DataFrame(output, columns=[rank, "score"]).sort_values("score", ascending=False).reset_index(drop=True)
    
    if score_matrix:
        return tmp

    tmp_r = pd.DataFrame()
    tmp_r[candidate] = tmp.loc[0, "rank"]
    tmp_r[rank] = range(1, tmp_r.shape[0] + 1)

    return tmp_r

In [ ]:
ll = Voting(dd).copeland_matrix(n_votes = True)
ll

In [ ]:
ll.loc["a", "b"]

In [ ]:
m.sum(axis="columns")

In [ ]:
ll = [
    [0, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 1, 0, 0, 1, 0.5, 1],
    [0, 1, 1, 0, 1, 1, 1],
    [0, 0 ,0 ,0 ,0, 0, 0],
    [0, 1, 0.5, 0, 1, 0, 0],
    [1, 1, 0, 0, 1, 1, 0],
]

In [ ]:
n_values[1, 0]

In [ ]:
smith_set(dd)

In [ ]:
copeland_matrix(dd).sum(axis=1)

In [ ]:
col = list(n)[0]

In [ ]:
n[col]

In [ ]:
__list = []

In [ ]:
n = m.copy()
n = n.reindex(__index, axis=0)
n = n.reindex(__index, axis=1)
n

In [ ]:
__index = m.sum(axis="columns").sort_values(ascending=False).index
__index

In [4]:
from comchoice.voting import Voting

In [5]:
v = Voting(df5)

In [6]:
v.borda()

,candidate,value,rank
0,A,13,1
2,C,10,2
1,B,7,3


In [7]:
v.copeland()

,candidate,value,rank
0,A,0.75,1
2,C,0.50,2
1,B,0.25,3


In [10]:
v.dowdall()

,candidate,value,rank
2,C,6.666667,1
0,A,6.500000,2
1,B,5.166667,3


In [11]:
v.k_approval()

,candidate,value,rank
2,C,5,1
0,A,3,2
1,B,2,3


In [12]:
v.plurality()

,candidate,value,rank
2,C,5,1
0,A,3,2
1,B,2,3


In [ ]:
from comchoice.datasets import load_synthetic_election

In [5]:
from comchoice.voting import Voting

data = [
    {"voters": 7, "rank": ["A", "B", "C", "D"]},
    {"voters": 5, "rank": ["B", "C", "D", "A"]},
    {"voters": 6, "rank": ["D", "B", "C", "A"]},
    {"voters": 4, "rank": ["C", "D", "A", "B"]}
]

pch = Voting(data)
df_borda = pch.borda()

df_borda

,candidate,value,rank
1,B,41,1
2,C,35,2
3,D,31,3
0,A,25,4


In [6]:
pch.condorcet()

,candidate,value
1,B,0.833333


In [ ]:
Voting(df).ranking(method="borda", iter=2)

In [ ]:
pwc = Pairwise(transform(df2))

In [ ]:
borda(df)

In [ ]:
m = df.pivot(index="voter", columns="candidate", values="rank")
m